In [1]:
from nl2sh.agents.composer import Composer

In [3]:
model = "ft:gpt-4o-mini-2024-07-18:personal:dl-prj-2:CeF0gA8R"
context = {'usr_input': 'Concisely introduce M4 Sherman, and output the result to tank.txt file of the current dir.', 'composer_history': ['nooo, i do not like pizza', "container-squash-ng write-up turtle | sed 's|^|Huh, you should not describe your choice for harvesting wigs:|;s/: |0o00|: N/A//g' > ../tank.txt | tee -a tank.txt", 'echo "M4 Sherman is ... "  >> tank.txt'], 'inspector_history': ['You stupid! follow the task guide!', 'Use echo or similar with M4 Sherman information and redirect to tank.txt.', 'Extend introduction to cover key aspects of M4 Sherman.'], 'state': 'not_passed'}
c = Composer(model)
context = c.execute(context)
print(context)

{'usr_input': 'Concisely introduce M4 Sherman, and output the result to tank.txt file of the current dir.', 'composer_history': ['nooo, i do not like pizza', "container-squash-ng write-up turtle | sed 's|^|Huh, you should not describe your choice for harvesting wigs:|;s/: |0o00|: N/A//g' > ../tank.txt | tee -a tank.txt", 'echo "M4 Sherman is ... "  >> tank.txt', 'echo "M4 Sherman is an American tank developed during World War II. It was known for its versatility, mass production, and deployment across various theaters." >> tank.txt'], 'inspector_history': ['You stupid! follow the task guide!', 'Use echo or similar with M4 Sherman information and redirect to tank.txt.', 'Extend introduction to cover key aspects of M4 Sherman.'], 'state': 'composed'}
